<font size="20"> <b>Start: -</b> </font>

Like all the programs, we start this program too by importing all the necessary classes. Since queue is used as a common shared buffer in producer-consuer architecture, we have to implement the queue module. Other modules that are required to be implemented include the requests, csv and threading.

In [1]:
import requests

import threading

from requests import Session

from queue import Queue

import csv

import time

<font size="20"> <b>Step 1: -</b> </font>

After all the necessary imports, we start off with the OOP Concepts implement the task in the code.

In [2]:
class GitHub_API_Task:
    
    def __init__(self):
        self.session = Session()
        self.details_of_user = []
        self.lock = threading.Lock( )
        
    def fetch_user_details(self, username):
        url = f"https://api.github.com/users/{username}"
        retries = 3
        for r in range(retries):
            try:
                response = self.session.get(url)
                response.raise_for_status()
                if response.status_code == 200:
                    user_data = response.json()
                    with self.lock:
                        self.details_of_user.append(user_data)
                    break  # Success, exit the retry loop
            except requests.exceptions.RequestException:
                time.sleep(120)
                pass                    # Ignore the exception and retry after 2 minutes
        else:
            print(f"Failed to fetch user details for {username} after {retries} attempts.")
                
    def saving_the_user_details(self,filename):
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['Username', 'Name', 'Location', 'Email', 'Public Repos']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for user_data in self.details_of_user:
                writer.writerow({
                    'Username': user_data['login'],
                    'Name': user_data['name'],
                    'Location': user_data['location'],
                    'Email': user_data['email'],
                    'Public Repos': user_data['public_repos']
                })

<font size="20"> <b>Code Description: -</b> </font>

__The above code snippent does the following: -__

1) Creates a class named as GitHub_API_Task.

2) Method __init__ defines the properties that would be defined inside the class. The method acts as a constructor for class.

<b>P.S. Code "self.lock = thread.Lock( )" initializes an instance of the Lock class and assigns it to the self.lock attribute of the class. The purpose of using self.lock is to ensure that the self.user_details list is accessed and modified in a thread-safe manner. By acquiring the lock using with self.lock, the code ensures that only one thread can access and modify the list at a time.</b> 

3) The method fetch_user_details takes the username as a parameter and gets the values from the users.

<b>P.S. By using with self.lock, the code acquires the lock before accessing and modifying the list, and releases the lock automatically when the with block is exited. This ensures that only one thread can access and modify the list at a time, preventing potential conflicts and ensuring data integrity.</b>

4) The method saving_the_user_details works by opening a file with "filename" as csv file and writes the fieldname as fieldnames for the csv files. The writerow then writes ({ 'Username': user_data['login'],'Name': user_data['name'],'Location': user_data['location'],'Email': user_data['email'],'Public Repos': user_data['public_repos']}) into the csv files as values.

In [3]:
def user_details_worker(usernames_queue, User):
    while True:
        username = usernames_queue.get()
        if username is None:
            break
        User.fetch_user_details(username)
        usernames_queue.task_done()

The __usernames_queue.task_done()__ code effectively notifies the queue that a task has been finished, which can be useful when coordinating multiple worker threads or processes that are processing items from the queue. The above function takes usernames_queue which is the shared buffer for producer and consumer and the User object as the parameters. Usernames are fetched from queue using __username = usernames_queue.get()__ code. 

In [4]:
# Create an instance of the Class

User = GitHub_API_Task()

# Create a queue to hold the usernames

usernames_queue = Queue()

In [5]:
# Create and start multiple producer threads

num_threads = 10
threads = []
for tds in range(num_threads):
    thread = threading.Thread(target=user_details_worker, args=(usernames_queue, User))
    thread.start()
    threads.append(thread)

<font size="20"> <b>Step 2: - </b> </font>

The given code creates multiple threads to execute the user_details_worker function concurrently. A loop is used to create and start the threads. It iterates num_threads times. In each iteration, a new threading.Thread object is created, specifying user_details_worker as the target function and (usernames_queue, fetcher) as the arguments passed to the user_details_worker function. The target parameter specifies the function to be executed by the thread. The args parameter is a tuple of arguments to be passed to the target function. The start() method is called on the thread object to start its execution. Each thread will begin executing the user_details_worker function with the provided arguments. The thread object is appended to the threads list, allowing you to keep track of all the created threads.

In [6]:
# Enqueue the usernames
usernames = ["eugeneyan", "DataTalksClub", "airbytehq", "great-expectations", "growthbook", "cloudquery", "whoiskatrin", "aws", "treeverse", "adilkhash"]
for username in usernames:
    usernames_queue.put(username)

# Wait for all tasks to be completed
usernames_queue.join()

In [7]:
# Stop the worker threads
for i in range(num_threads):
    usernames_queue.put(None)
for j in threads:
    thread.join()

# Save the user details to a CSV file
User.saving_the_user_details("user_detail.csv")